In [ ]:
#首先安装paddlex
! pip install paddlex -i https://mirror.baidu.com/pypi/simple

# 实验名称：用paddleX高阶API完成目标检测任务

## 实验介绍：

百度开放的PaddleX就是其中一个AI开放平台。PaddleX简单易用，且功能强大，不但适合个人的开发实践，也适合企业应用。而paddleX有两种模式，一种是可视化端模式，这种模式可以让我们零代码的完成一个目标检测任务。paddleX的另一种模式是高阶API模式，这种模式需要我们编写少量代码，才能完成一个目标检测任务。  

本次实验，我们就来讲解一下如何调用PaddleX高阶API 来完成我们的目标检测项目。

## 实验目标

> - 掌握paddleX进行目标检测任务的工作流程
> - 掌握本实验的核心代码

## 实验内容

### 1.准备阶段

### 1.1 数据准备  

对于本次实验的数据集，存放位置在data目录下。我们已经提前完成了标注，标注格式为VOC格式，标注之后生成了两个文件夹：Annotations和JPEGImages。其中Annotations用来存储数据集的xml文件；JPEGImages用来存储数据集的图片。
 
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/ad198d705b3d42fb8e7f79468b9e0b90744712806ca54910a9171256bf853a9a" width=300></center>
<center>数据集结构</center>

PaddleX在训练前需要导入数据集，这就需要解析图片与xml标注文件的路径，因为PaddleX的高阶API对数据集的路径要求比较严格，所以我们首先要生成几个txt文件，来向paddleX明确图片与xml文件的路径，这样，数据集才能被paddleX解析并加载。

本次数据集一共两个类别：fire、smoke


In [3]:
# -*- coding: utf-8 -*-
'''
说明：
       本文件的作用是解析数据集中图片与xml文件的路径；
       运行这个文件会生成四个个txt文件：
            labels.txt用来存储数据集中瑕疵的各个类别名称；
            train_list.txt用来存储训练集的图片，及对应的xml文件路径；
            val_list.txt用来存储验证集的图片，及对应的xml文件路径；
            test_list.txt用来存储测试集的图片，及对应的xml文件路径；

'''


import os       # OS模块含有文件操作的功能，对数据集读取时会用到该模块。
import random   # 导入 random 模块，用于随时生成数据
random.seed(0)  # 当我们设置相同的seed，每次生成的随机数相同

num_class = ['fire','smoke']  #数据集中所有类名

xmlfilepath=r'./traffic_lights/Annotations'   #xmlfilepath存储Annotations文件夹的路径
saveBasePath=r"./traffic_lights/"             #saveBasePath用来存储生成的txt文件路径
trainval_percent=0.98          #train+val数据集占总数据集98%，test占总数据集2%（主要用于进一步拆分数据集，属于中间变量）
train_percent=0.8              #train占数据集80%

temp_xml = os.listdir(xmlfilepath)  #temp_xml存储Annotations文件夹里的文件
total_xml = []                      #total_xml存储后缀为xml的文件
for xml in temp_xml:                #对于Annotations文件夹里的所有文件，如果文件是xml文件，将文件加入到total_xml中
    if xml.endswith(".xml"):
        total_xml.append(xml)

num=len(total_xml)                  #num为xml文件的个数
list=range(num)                     #list为所有xml文件的序号（0，1，2，3，...，num)
tv=int(num*trainval_percent)        #tv为train+val数据集的个数
tr=int(num*train_percent)             #tr为tarin数据集的个数
trainval= random.sample(list,tv)    #trainval存储train数据集和val数据集的文件序号
train=random.sample(trainval,tr)    #train存储train数据集的文件序号
 
print("train and val size",tv)                                      #检查train+val数据集个数
print("train size",tr)                                              #检查train数据集个数
ftest = open(os.path.join(saveBasePath,'test_list.txt'), 'w')       #创建一个file对象来将test的数据名写入test_list.txt文件
ftrain = open(os.path.join(saveBasePath,'train_list.txt'), 'w')     #创建一个file对象来将train数据名写入train_list.txt文件
fval = open(os.path.join(saveBasePath,'val_list.txt'), 'w')         #创建一个file对象来将val数据名写入val_list.txt
flabel = open(os.path.join(saveBasePath,'labels.txt'), 'w') #创建一个file对象来将label数据名写入labels.txt
for i in num_class:                                         #遍历所有的label名称
    flabel.write(i + "\n")                                  #将名称写入文件
flabel.close()                                              #关闭labels.txt文件

for i  in list:                         #遍历所有xml文件
    name=total_xml[i][:-4]+'\n'         #name存储这一行文件内容
    if i in trainval:                   #该文件在train+val数据集中，进一步检测是train数据集还是val数据集
        if i in train:                  #该文件在train数据集中，将其写入train_list.txt文件
            ftrain.write("Images/" + name.strip().split(".")[0] + ".jpg" + " " + "Annotations/" + name.strip().split(".")[0] + ".xml" + "\n")
        else:                           #该文件在val数据集中，将其写入val_list.txt文件 
            fval.write("Images/" + name.strip().split(".")[0] + ".jpg" + " " + "Annotations/" + name.strip().split(".")[0] + ".xml" + "\n")
    else:                               #该文件在test数据集中，将其写入test_list.txt文件
        ftest.write("Images/" + name.strip().split(".")[0] + ".jpg" + " " + "Annotations/" + name.strip().split(".")[0] + ".xml" + "\n")
             
ftrain.close()          #关闭train_list.txt文件
fval.close()            #关闭val_list.txt文件
ftest .close()          #关闭test_list.txt文件

train and val size 437
train size 356


运行上面的代码后，就会生成四个txt文件：  

> - labels.txt用来存储数据集中瑕疵的各个类别名称；
> - train_list.txt用来存储训练集的图片，及对应的xml文件路径；
> - val_list.txt用来存储验证集的图片，及对应的xml文件路径；
> - test_list.txt用来存储测试集的图片，及对应的xml文件路径；  


通过这四个文件，数据集已经被分成了训练集（train）、验证集（val）、测试集（test）。


### 1.2 数据预处理  

为了训练出一个效果更好的模型，在数据集加载进网络之前，一般要对数据进行预处理，比如改变一下输入图片的尺寸，使得图片更适合网络的运算。或者，我们也可以用一些数据增强技术，来扩充我们的数据集（说明：对于数据增强的原理，我们会在后面课程中详细讲解）。paddleX中定义了一个图像处理流程transforms，在transforms中，我们可以定义数据集的预处理方式，也可以定义数据增强的方式，使用起来非常方便。如下代码可实现数据预处理：


In [4]:
# -*- coding: utf-8 -*-
'''
说明：
       本文件展示了paddleX高阶API的使用过程；
       本文件通过调用paddleX高阶API可以实现数据导入、模型训练、模型预测等工作；
   
'''

# 导入matplotlib绘图库
import matplotlib
#在导入matplotlib库后，且在matplotlib.pyplot库被导入前加“matplotlib.use(‘agg’)”语句，可以使得在PyCharm中不显示绘图。
matplotlib.use('Agg') 

# OS模块含有文件操作的功能，对数据集读取时会用到该模块。
import os
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#导入paddleX的高阶API
import paddlex as pdx

#定义图像处理流程transforms
#定义训练数据的预处理方式，为了模型效果更好，可以对训练数据进行数据增强
#如下代码中，训练过程使用了MixupImage、RandomDistort、RandomExpand、RandomCrop、Resize和RandomHorizontalFlip共6种数据增强方式

#from paddlex.det import transforms #这是paddlex 1.3的API
from paddlex import transforms #这是paddlex 2的API

#对目标检测任务的数据进行操作。可以利用Compose类将图像预处理/增强操作进行组合
train_transforms = transforms.Compose([
    ##对图像进行mixup操作，可以使模型训练时数据得以增强，目前仅YOLOv3模型支持该操作
    #在前mixup_epoch轮使用mixup增强操作；当该参数为-1时，该策略不会生效。默认为-1
    #transforms.MixupImage(mixup_epoch=250), 
    #以一定的概率对图像进行随机像素内容变换，以达到数据增强的目的
    transforms.RandomDistort(),
    #随机扩张图像，以达到数据增强的目的
    transforms.RandomExpand(),
    #随机裁剪图像，以达到数据增强的目的
    transforms.RandomCrop(),
    #调整图像大小（resize）
    #target_size 表示 短边目标长度。默认为608。
    transforms.Resize(target_size=608, interp='RANDOM'),
    #以一定的概率对图像进行随机水平翻转，以达到数据增强的目的
    transforms.RandomHorizontalFlip(),
    #对图像进行标准化
    transforms.Normalize(), 
])


[05-14 09:57:28 MainThread @utils.py:79] WRN paddlepaddle version: 2.3.0-rc0. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
2022-05-14 09:57:29,336-WARNING: type object 'QuantizationTransformPass' has no attribute '_supported_quantizable_op_type'
2022-05-14 09:57:29,338-WARNING: If you want to use training-aware and post-training quantization, please use Paddle >= 1.8.4 or develop version


上面的代码中，对于训练数据集，我们使用了MixupImage、RandomDistort、RandomExpand、RandomCrop和RandomHorizontalFlip共5种数据增强方式。我们利用Compose类将图像预处理/增强操作进行了组合。  

同样的道理，对于验证集，我们也可以transforms.Compose()来进行数据的预处理，但是验证数据不需要数据增强，所以我们只需要改变一下图片的尺寸，把图片变成608×608大小，这个尺寸特别适合yolov3网络模型的运算。并且，我们也要对验证集的图片进行标准化操作。因此，去除数据增强相关代码后的代码如下。


In [5]:
#定义验证集的预处理方式，验证数据不需要数据增强
eval_transforms = transforms.Compose([
    #调整图像大小（resize）
    #target_size 表示 短边目标长度。默认为608。
    transforms.Resize(target_size=608, interp='CUBIC'),
    #对图像进行标准化
    transforms.Normalize(),
])



### 1.3 加载数据  

接下来，我们需要定义数据集Dataset，用来加载数据。在paddleX的高阶API中，目标检测可使用VOC和COCO两种数据加载方式。由于我们的数据集为VOC格式标注的，因此采用pdx.datasets.VOCDetection来加载训练数据集。
同样的道理，我们也可以用pdx.datasets.VOCDetection来加载验证数据，代码如下：



In [6]:
#定义数据集Dataset，用来加载数据
#目标检测可使用VOCDetection格式和COCODetection两种数据集，
#此处由于我们的数据集为VOC格式，因此采用pdx.datasets.VOCDetection来加载数据集，
train_dataset = pdx.datasets.VOCDetection(    
    data_dir='traffic_lights',                 #数据集路径
    file_list='traffic_lights/train_list.txt', #指向train_list.txt的路径，也即是训练集的路径
    label_list='traffic_lights/labels.txt',    #指向labels.txt的路径
    transforms=train_transforms,
    shuffle=True)

    
eval_dataset = pdx.datasets.VOCDetection(
    data_dir='traffic_lights',
    file_list='traffic_lights/val_list.txt', #指向val_list.txt的路径，也即是验证集的路径
    label_list='traffic_lights/labels.txt',  #指向labels.txt的路径
    transforms=eval_transforms)


2022-05-14 09:57:34 [INFO]	Starting to read file list from dataset...
2022-05-14 09:57:34 [INFO]	356 samples in file traffic_lights/train_list.txt, including 356 positive samples and 0 negative samples.
creating index...
index created!
2022-05-14 09:57:34 [INFO]	Starting to read file list from dataset...
2022-05-14 09:57:34 [INFO]	81 samples in file traffic_lights/val_list.txt, including 81 positive samples and 0 negative samples.
creating index...
index created!


**注意：**

上面代码中buffer_size表示数据集中样本在预处理过程中队列的缓存长度，这个值如果设置的过大，则会出现memory manager的问题，程序会报错。所以这个值不能设置过大。


### 2. 训练与评估  

准备阶段的工作完成之后，我们就可以进行模型的训练了， 代码如下。


In [7]:
#模型训练    
#模型训练过程每间隔save_interval_epochs轮会保存一次模型在save_dir目录下，
#同时在保存的过程中也会在验证数据集上计算相关指标
num_classes = len(train_dataset.labels) #num_classes表示瑕疵的类别数
#调用paddleX高阶API中的yolov3算法，其backbone为DarkNet53
model = pdx.det.YOLOv3(num_classes=num_classes, backbone='DarkNet53')   
#model = pdx.det.YOLOv3(num_classes=num_classes, backbone='MobileNetV1')

model.train(    
    num_epochs=100, #训练的epoch数
    train_dataset=train_dataset,    #加载增强后的训练数据
    train_batch_size=16,             #每一批次大小
    eval_dataset=eval_dataset,      #导入验证数据
    learning_rate=0.000125,         #学习率
    lr_decay_epochs=[50, 150],     #采用可变学习率，在这里规定学习率的范围，把epoch分为0-100、100-20、200-以上  三个范围，每个范围内采用不同的学习率
    pretrain_weights='COCO',
    save_interval_epochs=15,        #每隔多少epoch保存一次模型
    save_dir='saved_model', #模型的保存位置
    use_vdl=True)



W0514 09:57:34.591156   246 gpu_context.cc:244] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0514 09:57:34.596359   246 gpu_context.cc:272] device: 0, cuDNN Version: 7.6.


2022-05-14 09:57:38 [INFO]	Loading pretrained model from saved_model/pretrain/yolov3_darknet53_270e_coco.pdparams
2022-05-14 09:57:38 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.weight doesn't match.(Pretrained: [255, 1024, 1, 1], Actual: [21, 1024, 1, 1])
2022-05-14 09:57:38 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.bias doesn't match.(Pretrained: [255], Actual: [21])
2022-05-14 09:57:38 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.weight doesn't match.(Pretrained: [255, 512, 1, 1], Actual: [21, 512, 1, 1])
2022-05-14 09:57:38 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.bias doesn't match.(Pretrained: [255], Actual: [21])
2022-05-14 09:57:38 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.2.weight doesn't match.(Pretrained: [255, 256, 1, 1], Actual: [21, 256, 1, 1])
2022-05-14 09:57:38 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.2.bias doesn't match.

Exception in thread Thread-33:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 527, in _thread_loop
    batch = self._get_data()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 664, in _get_data
    batch.reraise()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/worker.py", line 169, in reraise
    raise self.exc_type(msg)
ValueError: DataLoader worker(0) caught ValueError with message:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/

KeyboardInterrupt: 

In [ ]:
model.evaluate(
    eval_dataset = eval_dataset, 
    batch_size=1
    )

> - 以上代码中，通过调用pdx.det.YOLOv3来构建一个基于YOLOv3算法的检测器。也就是说，在paddleX中，我们不需要再自己搭建yolov3算法的网络结构了，paddleX的API会帮我们搭建好。

> - pdx.det.YOLOv3有一些参数需要注意。num_classes不需要包含背景类，比如咱们本次实验的待检测物体共3类，则num_classes设为3即可，这里与FasterRCNN/MaskRCNN有差别。


> - lr_decay_epochs=[100, 200]表示我们采用可变学习率。[100, 200]规定了学习率的范围，表示把epoch分为“0-100、100-200、200-以上”三个范围，每个范围内采用不同的学习率。比如本次代码中，我们在0-100个epoch内，使用的learning_rate=0.000125，在100-200个epoch内，是学习率衰减0.1倍，变成0.0000125，当200个epoch以上时，学习率再次衰减0.1倍。


> - 以上代码中，模型训练过程每间隔save_interval_epochs轮会保存一次模型在save_dir目录下，同时在保存的过程中也会在验证数据集上计算相关指标。训练过程会持续十几个小时，当模型评估的loss值不再减小，或者变化不大时，就可以停止训练了。训练过程的轮次epoch、损失值loss，学习率lr等信息变化，如图所示。
 
 <center><img src="https://ai-studio-static-online.cdn.bcebos.com/877fc6ca58ea49619891a6f1056f469b375e43f2a4aa48b996ce5835808c4efe" width=600></center>
<center></center>
 
可以看到，随着训练的加深，loss值逐渐在减小。
模型训练结束后，通过语句eval_dataset=eval_dataset把验证集加载进模型，可以进行模型的评估。并且把评估结果输出在屏幕上，如下图所示。

 <center><img src="https://ai-studio-static-online.cdn.bcebos.com/585ff8593a004631b7e50e32beebdfc9ed242bcade714775999b622e83ee745e" width=600></center>
<center></center>
从图中最后一行代码，可以读出，在验证集上，当前最好的模型是epoch_10，预测框的mAP值为4.12。评估后，会把效果最好的模型保存下来，名字为best_model。


### 3. 预测展示
经过漫长的模型训练和评估之后，我们保存好一个效果不错的模型，就可以用这个模型进行预测，并把预测结果可视化的展示出来。预测部份代码如下。


In [ ]:
#预测阶段    
import paddlex as pdx
#加载训练好的模型
model = pdx.load_model('saved_model/epoch_60')
#选择待预测的图片

#image_name = 'traffic_lights/Images/red00154.jpg'
#image_name = 'traffic_lights/Images/red00208.jpg'
#image_name = 'traffic_lights/Images/red00090.jpg'
#image_name = 'traffic_lights/Images/green00082.jpg'
#image_name = 'traffic_lights/Images/red00173.jpg'
image_name = 'traffic_lights/Images/red00013.jpg'
#image_name = 'traffic_lights/Images/green00138.jpg'
#image_name = 'traffic_lights/Images/red00028.jpg'
#image_name = 'traffic_lights/Images/green00094.jpg'


#启动预测
result = model.predict(image_name)
#使用pdx.det.visualize将结果可视化，可视化结果将保存到save_dir目录下，
#其中threshold代表Box的置信度阈值，将Box置信度低于该阈值的框过滤掉，不进行可视化。
pdx.det.visualize(image_name, result, threshold=0.5, save_dir='./saved_model')



以上代码中，我们使用pdx.det.visualize将预测的结果可视化展示出来，可视化结果是一张jpg图片，程序会将该图片保存到save_dir目录下。其中参数threshold代表Box的置信度阈值， 如果模型预测出的矩形框的置信度很低，低于这个threshold值，则这个预测框不会展示出来。